# Python for Geospatial Analysis - Final Assignment
### _Maria Francisca Archila Bustos_

Import libraries to use:

In [ ]:
import os
import rasterio
import numpy as np
import pandas as pd
import urllib.request
import zipfile
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas as gpd
from matplotlib.ticker import LogFormatterExponent
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pysal.lib import weights
from pysal.explore import esda

%matplotlib inline

## Raster analysis preparations

Create a couple of variables that we will use throughout:

In [ ]:
urbanFolder = "Data/SSP4_Inequality/SSP4/Urban/GeoTIFF"
totalFolder = "Data/SSP4_Inequality/SSP4/Total/GeoTIFF"

Fetch data for later use:

In [ ]:
cntrys = rasterio.open("Data/countries.tif").read(1)

Take a look at one dataset (total 2010)

In [ ]:
# Open it
pop2010_tif = rasterio.open(totalFolder +  '/ssp4_2010.tif')
pop2010_array = pop2010_tif.read(1)

# Plot
plt.figure(figsize=(14, 14))
ax = plt.gca()
imgplot = plt.imshow(pop2010_array, norm=colors.LogNorm(), cmap='Greys')

# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
# https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(imgplot, cax=cax)

# Get summary statistics
print("Mean population:","{0:,.0f}".format(np.mean(pop2010_array)))
print("Minimum population:","{0:,.0f}".format(np.min(pop2010_array, axis=None)))
print("Maximum population:","{0:,.0f}".format(np.max(pop2010_array, axis=None)))
print("Total population:","{0:,.0f}".format(np.sum(pop2010_array, axis=None, dtype=np.int64)))

In [ ]:
pop2010_array

It seems that no data cells (water) are set to 2147483647. Change these to zero and re-run.

In [ ]:
pop2010_array_fx = np.copy(pop2010_array)
pop2010_array_fx[pop2010_array_fx == 2147483647] = 0 

# Plot
plt.figure(figsize=(14, 14))
ax = plt.gca()
imgplot = plt.imshow(pop2010_array_fx, norm=colors.LogNorm(), cmap='Greys')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(imgplot, cax=cax)

# Get summary statistics
print("Mean population:","{0:,.0f}".format(np.mean(pop2010_array_fx)))
print("Minimum population:","{0:,.0f}".format(np.min(pop2010_array_fx, axis=None)))
print("Maximum population:","{0:,.0f}".format(np.max(pop2010_array_fx, axis=None)))
print("Total population:","{0:,.0f}".format(np.sum(pop2010_array_fx, axis=None, dtype=np.int64)))

Take a look at the country dataset.

In [ ]:
# Plot
plt.figure(figsize=(14, 14))
ax = plt.gca()
imgplot = plt.imshow(cntrys, cmap='Paired')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(imgplot, cax=cax)

# Get summary statistics
print("Minimum country ID:","{0:f}".format(np.min(cntrys, axis=None)))
print("Maximum country ID:","{0:f}".format(np.max(cntrys, axis=None)))

In [ ]:
cntrys

No data cells are set to -99.

## Raster analysis # 1
Write a function that plots the projected total and urban population from 2010 to 2100 for one country in a line chart.

In [ ]:
def projectedPopLine (totalFolder, urbanFolder, countryData, countryID):
    urbPopList = []
    totPopList = []
    yrList = []
    cntry = countryData == countryID
    
    for yr in range(2010,2110,10):
        # Get the files
        tot = rasterio.open(totalFolder +  '/ssp4_' + str(yr) + '.tif').read(1)
        urb = rasterio.open(urbanFolder +  '/ssp4urb' + str(yr) + '.tif').read(1)
        
        # Clean them up by removing no data value
        tot[tot == 2147483647] = 0
        urb[urb == 2147483647] = 0
        
        # Get the urban and total population for the selected country and add to the list (in millions)
        yrList.append(yr)
        totPopList.append(np.sum(tot[cntry])/1000000)
        urbPopList.append(np.sum(urb[cntry])/1000000)
    
    # Create a dataframe of the population per year
    pop = {'Total population': totPopList, 'Urban population': urbPopList}
    pop_df = pd.DataFrame(data = pop, index = yrList)
    
    # Plot on a line chart
    chart = pop_df.plot(y=['Total population', 'Urban population'], 
                        title = 'Projected total and urban population \n in country with ISO: ' + str(countryID),
                        rot = 90)
    
    # Add labels and format
    chart.set_xlabel('Year')
    chart.set_ylabel('Population (millions)')
    chart.ticklabel_format(axis='y', style='plain')
    
    return chart

Plot the total and urban population for Canada (ISO: 124)


In [ ]:
output = projectedPopLine(totalFolder, urbanFolder, cntrys, 124)

## Raster analysis # 2
Generate a global raster that shows only the cells that are projected to lose population, and indicates how much the population in those cells is projected to decline between 2010 and 2100.

In [ ]:
# Get the two datasets
pop2010 = rasterio.open(totalFolder +  '/ssp4_2010.tif').read(1)
pop2100 = rasterio.open(totalFolder +  '/ssp4_2100.tif').read(1)

# Clean them up by removing no data value
pop2010c = np.copy(pop2010)
pop2100c = np.copy(pop2100)
pop2010c[pop2010c == 2147483647] = 0
pop2100c[pop2100c == 2147483647] = 0
        
# Get the population change
popChange = pop2100c - pop2010c

# Convert to float so we can set NaN
popChange = popChange.astype(dtype=float)

# Get the declining population
popChange[popChange >= 0] = np.nan

# Multiply by -1 to be able to plot log
popDecline = popChange * -1

# Create background colour ramp (https://stackoverflow.com/questions/9707676/defining-a-discrete-colormap-for-imshow-in-matplotlib)
cmap = colors.ListedColormap(['white', '#f0f0f0'])
bounds=[-100,0,900]
norm = colors.BoundaryNorm(bounds, cmap.N)

# Plot the figure
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(16,8), gridspec_kw={'height_ratios': [30, 1]})
ax1.set_title('Projected global population decline between 2010 and 2100')
ax1.axis("off")

img1 = ax1.imshow(cntrys, cmap=cmap, norm=norm)
img2 = ax1.imshow(popDecline, norm=colors.LogNorm(), cmap='GnBu')

cb_label = ("Log of population decline (number of people less in 2100 as compared to 2010)"
        "\n  \n Notes: Population decline ranges from {:,.0f} to {:,.0f} people per cell during this period"
        "\n Areas in grey are places which are projected to experience growth or no change".format(np.nanmin(popDecline),np.nanmax(popDecline))
        )
cb = fig.colorbar(img2, cax=ax2, orientation= "horizontal", format = '%1.0f') 
cb.set_label(cb_label)

plt.show()

## Spatial autocorrelation preparations

Get the required data:

In [ ]:
# World shapefile
nd_url = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip"
    
if os.path.isfile('Data/ne_110m_admin_0_countries.shp'):
    print("World file already downloaded and unzipped.")
else:
    urllib.request.urlretrieve(nd_url, "NatureData.zip")
    zf = zipfile.ZipFile("NatureData.zip")
    zf.extractall(path = 'Data/')
    zf.close()
    os.remove("NatureData.zip") # clean up
    print("Download and unzip complete.")

world = gpd.read_file("Data/ne_110m_admin_0_countries.shp")

# Mortality csv
mortality = pd.read_csv('Data/under5mortality.csv')

Examine the data:

In [ ]:
world.head()

In [ ]:
world.columns


In [ ]:
world.loc[:, ['ISO_A3']].dtypes

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
worldplot = world.plot(ax=ax)

In [ ]:
mortality.head()

In [ ]:
mortality.columns

In [ ]:
mortality.loc[:,['ISO']].dtypes

## Spatial autocorrelation # 1
Perform a left join between the world shapefile and the child mortality rates.

_A left join will keep all countries, regardless of whether or not they have a reported child mortality rate. Countries with no reported child mortality rate will be given a null (NaN) value._

In [ ]:
# Keep only relevant columns
world_rel = world.loc[:,['ISO_A3','NAME_EN', 'geometry']]
mortality_rel= mortality.loc[:,['ISO','ChildMortality']]
mort_by_cntry = pd.merge(world_rel, mortality_rel, left_on='ISO_A3', right_on='ISO', how='left')
mort_by_cntry.columns

Examine the result:

In [ ]:
mort_by_cntry.head()

There are some countries with no child mortality data. Take a look at these.

In [ ]:
mort_by_cntry[mort_by_cntry['ChildMortality'].isnull()]

Most of these are small islands or countries where data may not be available. However, it is strange that Norway and France do not have ISO values and therefore did not participate in the join. The other countries with no ISO codes likely are not in the child mortality file. 

Take a look at the child mortality file for France and Norway.

In [ ]:
mortality[mortality['ISO'].isin(['FRA', 'NOR'])]

Both Norway and France have child mortality values. Take a look in the world data to determine if there are, perhaps, duplicate polygons for these countries.

In [ ]:
world[world['ISO_A3'].isin(['FRA', 'NOR'])]

Since Norway and France appear to exist only once in the data but without ISO codes, but they both have child mortality values, update these.

In [ ]:
# France
mort_by_cntry.loc[43,['ChildMortality']] = 4.2

# Norway
mort_by_cntry.loc[21,['ChildMortality']] = 2.6

mort_by_cntry.loc[[21,43]]

## Spatial autocorrelation # 2
Calculate the spatial weights matrix based on border neighbourhood and use the weights to calculate Moran's I for child mortality.

In [ ]:
# In order to calculate Moran's I NaN values are removed. 
# This means that the neighbourhood is calculated between adjacent neighbours with data.
mort_by_cntry_nonan = mort_by_cntry.dropna(subset=['ChildMortality'])
mort_by_cntry_nonan.reset_index(drop=True, inplace=True)

w_queen = weights.Queen.from_dataframe(mort_by_cntry_nonan)

# Show the weights matrix
pd.DataFrame(w_queen.full()[0], 
             index=mort_by_cntry_nonan['ISO_A3'],
             columns=mort_by_cntry_nonan['ISO_A3'],
            ).astype(int)


There are some disconnected observations. These are assumed to be actual islands that do not have any contiguous neighbourhood. Below we will check them.

In [ ]:
mort_by_cntry_nonan.loc[[0, 18, 41, 42, 73, 84, 129, 130, 131, 132, 137, 140, 148, 152, 164]]

The assumption was correct. We will thus ignore this warning and use our spatial weights matrix to calculate Moran's I. First, I want to take a look at the matrix as a map.

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.set_aspect('equal')
mort_by_cntry_nonan.plot(ax=ax, color='white', edgecolor='grey', linewidth=0.5)
w_queen.plot(mort_by_cntry_nonan, ax=ax, color='red')
plt.show()

Calculate Moran's I using the spatial weights matrix.

In [ ]:
moran = esda.Moran(mort_by_cntry_nonan['ChildMortality'], w_queen)

print("Moran's I for child mortality by country is {:0.3f} with a p-value of {:0.3f}".format(moran.I, moran.p_sim))